In [ ]:
import neo4j as neo
import os
from dotenv import load_dotenv
from collections import defaultdict

load_dotenv()


driver = neo.GraphDatabase.driver(os.getenv('NEO4J_URI'), auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD')))


def CoefJacc(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def find_similar_users_games(user_name):
    similar_users_games = defaultdict(set)

    with driver.session() as session:
        # Obtener los likes del Usario a evaluar
        user_categories = set(session.run(
            "MATCH (u:User {name: $user_name})-[:LIKES]->(:Reconode)-[:BELONGS_TO]->(c:Label) RETURN c.name",
            user_name=user_name
        ).values())

        # Buscar Similitudes
        similar_users = session.run(
            "MATCH (u:User)-[:LIKES]->(:Reconode)-[:BELONGS_TO]->(c:Label) WHERE u.name <> $user_name AND c.name IN $user_categories RETURN u.name",
            user_name=user_name,
            user_categories=list(user_categories)
        )

        # Ecnontrada las similitudes de usuario, buscar las similitudes de juego
        for record in similar_users:
            similar_user_name = record["u.name"]

            similar_user_games = set(session.run(
                "MATCH (u:User {name: $user_name})-[:LIKES]->(g:Reconode) RETURN g.name",
                user_name=similar_user_name
            ).values())

       # Revisar si el coeficiente de Jaccard cumple con el > 0.75
            similarity = CoefJacc(user_categories, similar_user_games) 
            if similarity > 0.75:
                similar_users_games[similar_user_name] = similar_user_games

    return similar_users_games